In [ ]:
# set Keras backend 
import os 
os.environ["KERAS_BACKEND"] = "torch"

# set warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# import modules and components
from FEXT.commons.utils.dataloader.generators import training_data_pipeline
from FEXT.commons.utils.dataloader.serializer import DataSerializer, ModelSerializer
from FEXT.commons.utils.models.inferencer import FeatureExtractor
from FEXT.commons.utils.validation.model import ModelValidation
from FEXT.commons.constants import CONFIG
from FEXT.commons.logger import logger

# 1. Load data and model

In [ ]:
# selected and load the pretrained model, then print the summary 
modelserializer = ModelSerializer()         
model, parameters = modelserializer.load_pretrained_model()
model_folder = modelserializer.loaded_model_folder
model.summary(expand_nested=True)

# isolate the encoder from the autoencoder model, and use it for inference     
extractor = FeatureExtractor(model)
encoder_model = extractor.encoder_model

### 1.1 Create generator and datasets

In [ ]:
# extract paths
dataserializer = DataSerializer()
train_data, validation_data = dataserializer.load_preprocessed_data(model_folder)

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators    
eval_batch_size = CONFIG["evaluation"]["BATCH_SIZE"]
train_dataset, validation_dataset = training_data_pipeline(train_data, validation_data, batch_size=eval_batch_size)

# 2. Model performance evaluation

### 2.1 Evaluation of loss and metrics

In [ ]:
validator = ModelValidation(model)
validator.evaluation_report(train_dataset, validation_dataset) 

### 2.2 Reconstruction evaluation

Compare reconstructed images to original pictures to qualitatively evaluate the performance of the FeXT autoencoder model

In [ ]:
# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators    
train_dataset, validation_dataset = training_data_pipeline(train_data, validation_data, batch_size=10)


# perform visual validation for the train dataset (initialize a validation tf.dataset
# with batch size of 10 images)
validator.visualize_reconstructed_images(train_dataset, 'visual_evaluation_train', model_folder)
validator.visualize_reconstructed_images(validation_dataset, 'visual_evaluation_val', model_folder)

Visualize the original image and the reconstructed images, together with the corresponding features vector. The raw vector has shape 4x4x512 and is reshaped to be 64x128

In [ ]:
train_dataset, validation_dataset = training_data_pipeline(train_data, validation_data, batch_size=10)
train_batch = train_dataset.take(1)

# extract features vector
recostructed_image = model.predict(train_batch, verbose=0) 
extracted_features = encoder_model.predict(train_batch, verbose=0)
reshaped_features = extracted_features.reshape(64, 128)

plot_name = 'visual_features_vector'
for original_image, label in train_batch:
    validator.visualize_features_vector(original_image, reshaped_features, recostructed_image, plot_name, model_folder)